# LOFAR single station imaging - Video making

In [ ]:
import getpass
import datetime
import numpy as np

import sys
sys.path.append('..')

from lofarimaging import get_station_type
from lofarimaging.rfi_tools import analyze_files, print_summary, measure_processing_duration
from lofarimaging.rfi_tools import generate_time_sweep, generate_subband_sweep, generate_height_sweep
from lofarimaging.rfi_tools import get_number_of_measurements_time_sweep, get_number_of_measurements_subband_sweep, get_number_of_measurements_height_sweep

In [ ]:
try:
    get_ipython().run_line_magic("load_ext", "autoreload")
    get_ipython().run_line_magic("autoreload", "2")
except NameError:
    pass

In [ ]:
import os

username = getpass.getuser()
data_dir = f"/home/{username}/Documents/LV614_image_data/"
caltable_dir = f"/home/{username}/Documents/LV614_image_data/CalTables/"
# List the contents of caltable_dir and print them
caltable_contents = os.listdir(caltable_dir)
print("Contents of caltable_dir:", caltable_contents)

output_dir = "../results/movie"
temp_dir = output_dir + "/temp"
fps = 10
short_sweep = True

# Observation dataset
obs_dir = data_dir + "sess_sid20230111T071252_LV614/scan_59955.30061/LV614_20230111_071256_spw3_sb51_461_int1_dur24300_dir0,0,SUN_xst/"

# Observation info
station_name = "LV614"
integration_time_s = 2
rcu_mode = 3
height = 1.5
station_type = get_station_type(station_name)


In [ ]:
df, summary = analyze_files(obs_dir)
print_summary(summary)

In [ ]:
# Time sweep of the selected subbands
height_time_sweep = 1.5
subbands_time_sweep = [110, 220, 330, 440]

# Subband sweep at a given time
height_subband_sweep = 1.5
subband_min = 52
subband_max = 460
subband_step = 1
subbands = df["subband"].unique()
subbands_subband_sweep = [s for s in sorted(subbands) if subband_min <= s <= subband_max][::subband_step]
min_time = df["timestamp"].min()
times_subband_sweep = [
    #min_time + datetime.timedelta(minutes=5),
    min_time + datetime.timedelta(minutes=50),
    min_time + datetime.timedelta(minutes=200),
]

# Height sweep at a given time and subband
height_min = 0
height_max = 1000
height_step = 1.5
heights_height_sweep = np.arange(height_min, height_max, height_step)
subbands_height_sweep = [110, 220, 330, 440]
min_time = df["timestamp"].min()
times_height_sweep = [
    #min_time + datetime.timedelta(minutes=5),
    min_time + datetime.timedelta(minutes=50),
    min_time + datetime.timedelta(minutes=200),
]

# Estimation of the processing duration
duration = measure_processing_duration(df=df, station_name=station_name, station_type=station_type, rcu_mode=rcu_mode, temp_dir=temp_dir)

time_sweep_measurements = get_number_of_measurements_time_sweep(df=df, subbands=subbands_time_sweep, short_sweep=short_sweep)
subband_sweep_measurements = get_number_of_measurements_subband_sweep(df=df, subbands=subbands_subband_sweep, times=times_subband_sweep, short_sweep=short_sweep)
height_sweep_measurements = get_number_of_measurements_height_sweep(df=df, subbands=subbands_height_sweep, times=times_height_sweep, heights=heights_height_sweep, short_sweep=short_sweep)

time_sweep_duration = datetime.timedelta(seconds=duration * time_sweep_measurements)
subband_sweep_duration = datetime.timedelta(seconds=duration * subband_sweep_measurements)
height_sweep_duration = datetime.timedelta(seconds=duration * height_sweep_measurements)



print(f"Estimated duration of the time sweep movie generation: {str(time_sweep_duration)}")
print(f"Estimated duration of the subband sweep movie generation: {str(subband_sweep_duration)}")
print(f"Estimated duration of the height sweep movie generation: {str(height_sweep_duration)}")
print(f"Total number of measurements: {time_sweep_measurements + subband_sweep_measurements + height_sweep_measurements}")
print(f"Estimated duration of the full movie generation: {str(time_sweep_duration + subband_sweep_duration + height_sweep_duration)}")

In [ ]:
generate_time_sweep(
    df=df,
    subbands=subbands_time_sweep,
    height=height_time_sweep,
    station_name=station_name,
    station_type=station_type,
    rcu_mode=rcu_mode,
    temp_dir=temp_dir,
    output_dir=output_dir,
    fps=fps,
    short_sweep=short_sweep,
)

In [ ]:
generate_subband_sweep(
    df=df,
    times=times_subband_sweep,
    subbands=subbands_subband_sweep,
    height=height_subband_sweep,
    station_name=station_name,
    station_type=station_type,
    rcu_mode=rcu_mode,
    temp_dir=temp_dir,
    output_dir=output_dir,
    fps=fps,
    short_sweep=short_sweep,
)

In [ ]:
generate_height_sweep(
    df=df,
    times=times_height_sweep,
    subbands=subbands_height_sweep,
    heights=heights_height_sweep,
    station_name=station_name,
    station_type=station_type,
    rcu_mode=rcu_mode,
    temp_dir=temp_dir,
    output_dir=output_dir,
    fps=fps,
    short_sweep=short_sweep,
)